In [1]:
library(fastDummies)
library(cplm)
library(tweedie)
library(statmod)
library(HDtweedie)
library(TDboost)
library(mgcv)
library(MASS)

Loading required package: coda

Loading required package: Matrix

Loading required package: splines

Loading required package: lattice

Loaded TDboost 1.2


Loading required package: nlme


Attaching package: 'nlme'


The following objects are masked from 'package:cplm':

    fixef, ranef, VarCorr


This is mgcv 1.8-38. For overview type 'help("mgcv-package")'.



In [2]:
x1 <-  as.factor(sample(c(1,2),size=5000,prob=c(0.4,0.6),replace=T))
sigma_matrix<- matrix(c(1,0.1,0.15,0.1,1,0.3,0.15,0.3,1),nrow=3,byrow=T)
X<- mvrnorm(n = 5000, mu=c(0,-1,1), Sigma=sigma_matrix, tol = 1e-6, empirical = FALSE, EISPACK = FALSE)
design_x_train <- as.data.frame(cbind(x1,X))
design_x_train[,1]<-as.factor(design_x_train[,1])
design_x <- model.matrix(~.,data=design_x_train)
beta_num<- rep(0.5,5)
mu<-exp(design_x%*%beta_num)
y_sim_train<- rTweedie(mu,p=1.7,phi=9)

In [3]:
length(which(y_sim_train==0))

[1] 3061

In [4]:
summary(y_sim_train)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.000   0.000   0.000   4.067   1.336 450.295 

In [5]:
sim_full_train<-cbind(design_x_train,y_sim_train)

In [6]:
sam <- sample(rep(1:2,len=5000))

In [7]:
sim_train<-sim_full_train[sam==1,]
sim_test<-sim_full_train[sam==2,]

In [8]:
length(which(sim_train$y==0))

[1] 1541

In [9]:
length(which(sim_test$y==0))

[1] 1520

In [10]:
design_x_strain<-design_x_train[sam==1,]
design_x_stest<-design_x_train[sam==2,]

In [11]:
design_x_strain$y=sim_train$y_sim_train

In [12]:
fit1<-glm(y~.,data=design_x_strain,family=tweedie(link.power=0,var.power=1.7))
summary(fit1)
mean(residuals(fit1)^2)


Call:
glm(formula = y ~ ., family = tweedie(link.power = 0, var.power = 1.7), 
    data = design_x_strain)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-4.530  -2.998  -2.541  -0.631   9.634  

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.32257    0.12567   2.567   0.0103 *  
x12          0.58414    0.10906   5.356 9.29e-08 ***
V2           0.53589    0.05562   9.634  < 2e-16 ***
V3           0.39543    0.05561   7.111 1.50e-12 ***
V4           0.56616    0.05499  10.296  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Tweedie family taken to be 9.346251)

    Null deviance: 22215  on 2499  degrees of freedom
Residual deviance: 18134  on 2495  degrees of freedom
AIC: NA

Number of Fisher Scoring iterations: 7


[1] 7.253751

In [13]:
head(design_x_strain)

,x1,V2,V3,V4,y
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
2,1,1.26392067,-1.6233876,2.1452697,2.509473e+01
4,2,-1.54173700,-0.3136212,2.2837124,0.000000e+00
5,1,-1.28855523,-1.3790889,-0.7575581,2.651409e-04
7,2,-0.37746269,-0.8951082,2.2420641,0.000000e+00
9,2,0.94558157,-2.8990485,-0.5039920,0.000000e+00
10,1,-0.07654452,0.0414496,1.8452422,1.210923e+00


In [14]:
autocross<-design_x_strain[,1:4]

In [15]:
ytrain= design_x_strain$y

In [16]:
result_round1=matrix(nrow=dim(autocross)[2],ncol=dim(autocross)[2])
for(i in 2:(dim(autocross)[2])-1){
    for(j in (i+1):(dim(autocross)[2])){
        if (is.numeric(autocross[,i])&&is.numeric(autocross[,j])){
            tmp_x <- autocross[,i]*autocross[,j]
            tmp_fit<-glm(ytrain~tmp_x+offset(log(fitted(fit1)))
                   ,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
            result_round1[i,j]=mean(residuals(tmp_fit)^2)
        }
        else if(is.factor(autocross[,i])&&is.factor(autocross[,j])){
            tmp_x<-model.matrix(~autocross[,i]*autocross[,j])
            tmp_x_x<-model.matrix(~autocross[,i]+autocross[,j])
            tmp_diff <- setdiff(colnames(tmp_x),colnames(tmp_x_x))
            tmp_x<-tmp_x[,tmp_diff]
            tmp_x<-as.matrix(tmp_x)
            if (ncol(tmp_x)<=5){
            tmp_fit<-glm(ytrain~tmp_x+offset(log(fitted(fit1)))
                   ,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
            result_round1[i,j]=mean(residuals(tmp_fit)^2)}
            else if (ncol(tmp_x)>5){
                result = random_categorical_data(X=tmp_x,y=ytrain,fitted_offset=fitted(fit1))
                result_round1[i,j] = result$'min Deviance'
            }}
        else{
            tmp_x<-model.matrix(~autocross[,i]:autocross[,j])
            tmp_x<-tmp_x[,-c(1,2)]
            tmp_fit<-glm(ytrain~tmp_x+offset(log(fitted(fit1)))
                   ,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
            result_round1[i,j]=mean(residuals(tmp_fit)^2)
    }
  }
}

In [17]:
round1_index <- which(result_round1 == min(result_round1,na.rm=T), arr.ind = TRUE)
round1_index

row,col
2,3


In [19]:
round1_fit<-glm(y~.+V2*V3,data=design_x_strain
                   ,family=tweedie(link.power=0,var.power=1.7),control=list(maxit=500))
summary(round1_fit)


Call:
glm(formula = y ~ . + V2 * V3, family = tweedie(link.power = 0, 
    var.power = 1.7), data = design_x_strain, control = list(maxit = 500))

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.6500  -2.9721  -2.5460  -0.6116   9.8328  

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  0.28064    0.12458   2.253   0.0244 *  
x12          0.58483    0.10709   5.461 5.20e-08 ***
V2           0.65735    0.07151   9.193  < 2e-16 ***
V3           0.37100    0.05536   6.702 2.54e-11 ***
V4           0.56304    0.05397  10.433  < 2e-16 ***
V2:V3        0.14377    0.05298   2.713   0.0067 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Tweedie family taken to be 8.989923)

    Null deviance: 22215  on 2499  degrees of freedom
Residual deviance: 18066  on 2494  degrees of freedom
AIC: NA

Number of Fisher Scoring iterations: 7


In [20]:
mean(residuals(round1_fit)^2)/mean(residuals(fit1)^2)

[1] 0.9962422

In [21]:
AICtweedie(fit1)

[1] 8688.177

In [22]:
AICtweedie(round1_fit)

[1] 8682.993

In [24]:
design_x_test_full<- model.matrix(~.,data=design_x_stest[,1:4])
design_x_stest$y <- sim_test$y_sim_train

In [25]:
pred_fit1<-exp(design_x_test_full%*%fit1$coefficients)

In [27]:
tmp_x<-model.matrix(~.+V2*V3,data=design_x_stest[,1:4])
pred_round1<-exp(tmp_x%*%round1_fit$coefficients)
mean(tweedie.dev(design_x_stest$y,pred_fit1,power=1.7))/mean(tweedie.dev(design_x_stest$y,pred_round1,power=1.7))

[1] 0.9945702

In [28]:
mean(tweedie.dev(design_x_stest$y,pred_round1,power=1.7))/mean(tweedie.dev(design_x_stest$y,pred_fit1,power=1.7))

[1] 1.005459